## Virginia Counties
#### Exploratory Data Analysis

https://www.kaggle.com/muonneutrino/us-census-demographic-data/download

In [1]:
import pandas as pd
import numpy as np
import os
import shapely
import shapefile
import geopandas
import plotly
from plotly.figure_factory._county_choropleth import create_choropleth
import xlrd

In [2]:
import plotly as py
import plotly.graph_objs as go
# from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
print('New York is for Python lovers!')

New York is for Python lovers!


In [4]:
# read in the dataset
counties = pd.read_csv('https://raw.githubusercontent.com/austinlasseter/dash-virginia-counties/master/resources/acs2017_county_data.csv')
counties.sample(3)

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
1319,27011,Minnesota,Big Stone County,5039,2489,2550,1.4,98.0,0.3,0.0,...,4.3,0.6,7.5,17.5,2388,70.7,17.0,11.9,0.3,2.8
1408,28015,Mississippi,Carroll County,10221,5157,5064,0.5,64.3,33.5,0.1,...,1.3,0.0,1.6,28.8,3737,71.9,18.3,9.3,0.5,7.0
1043,21101,Kentucky,Henderson County,46252,22409,23843,2.4,87.0,7.1,0.1,...,1.6,0.8,2.4,20.8,20942,82.3,13.1,4.4,0.1,4.8


In [5]:
# restrict to Virginia
ny = counties.loc[counties['State']=='New York']
print(counties.shape)
print(ny.shape)

(3220, 37)
(62, 37)


In [6]:
counties['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

In [7]:
# show a list of variables
counties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CountyId          3220 non-null   int64  
 1   State             3220 non-null   object 
 2   County            3220 non-null   object 
 3   TotalPop          3220 non-null   int64  
 4   Men               3220 non-null   int64  
 5   Women             3220 non-null   int64  
 6   Hispanic          3220 non-null   float64
 7   White             3220 non-null   float64
 8   Black             3220 non-null   float64
 9   Native            3220 non-null   float64
 10  Asian             3220 non-null   float64
 11  Pacific           3220 non-null   float64
 12  VotingAgeCitizen  3220 non-null   int64  
 13  Income            3220 non-null   int64  
 14  IncomeErr         3220 non-null   int64  
 15  IncomePerCap      3220 non-null   int64  
 16  IncomePerCapErr   3220 non-null   int64  


#### Data Dictionary
| Varname | Data | 
|:---:|:---|
| CensusTract | Census tract ID |
| State | State, DC, or Puerto Rico |
|County | County or county equivalent | 
| Total | PopTotal population | 
| Men | Number of men |
|Women|Number of women |
|Hispanic|% of population that is Hispanic/Latino|
|White|% of population that is white|
|Black|% of population that is black|
|Native|% of population that is Native American or Native Alaskan|
|Asian|% of population that is Asian|
|Pacific|% of population that is Native Hawaiian or Pacific Islander|
|Citizen|Number of citizens|
|Income|Median household income |
|IncomeErr|Median household income error|
|IncomePerCap|Income per capita |
|IncomePerCapErr|Income per capita error |
|Poverty|% under poverty level|
|ChildPoverty|% of children under poverty level|
|Professional|% employed in management, business, science, and arts|
|Service|% employed in service jobs|
|Office|% employed in sales and office jobs|
|Construction|% employed in natural resources, construction, and maintenance|
|Production|% employed in production, transportation, and material movement|
|Drive|% commuting alone in a car, van, or truck|
|Carpool|% carpooling in a car, van, or truck|
|Transit|% commuting on public transportation|
|Walk|% walking to work|
|OtherTransp|% commuting via other means|
|WorkAtHome|% working at home|
|MeanCommute|Mean commute time (minutes)|
|Employed|Number of employed (16+)|
|PrivateWork|% employed in private industry|
|PublicWork|% employed in public jobs|
|SelfEmployed|% self-employed|
|FamilyWork|% in unpaid family work|
|Unemployment|Unemployment rate (%)|

In [8]:
counties.columns

Index(['CountyId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

### Rural or Urban?
https://www.ers.usda.gov/data-products/rural-urban-continuum-codes/

In [9]:
# get the USDA county rural-urban codes
usda=pd.read_excel('https://github.com/austinlasseter/dash-virginia-counties/raw/master/resources/ruralurbancodes2013.xls')
usda.sample(3)

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description
1154,22083,LA,Richland Parish,20725,6,"Nonmetro - Urban population of 2,500 to 19,999..."
294,8099,CO,Prowers County,12551,7,"Nonmetro - Urban population of 2,500 to 19,999..."
864,19151,IA,Pocahontas County,7310,9,"Nonmetro - Completely rural or less than 2,500..."


In [10]:
# what are the codes?
usda.groupby('RUCC_2013')[['RUCC_2013','Description']].max()

,RUCC_2013,Description
RUCC_2013,,
1,1,Metro - Counties in metro areas of 1 million p...
2,2,"Metro - Counties in metro areas of 250,000 to ..."
3,3,Metro - Counties in metro areas of fewer than ...
4,4,"Nonmetro - Urban population of 20,000 or more,..."
5,5,"Nonmetro - Urban population of 20,000 or more,..."
6,6,"Nonmetro - Urban population of 2,500 to 19,999..."
7,7,"Nonmetro - Urban population of 2,500 to 19,999..."
8,8,"Nonmetro - Completely rural or less than 2,500..."
9,9,"Nonmetro - Completely rural or less than 2,500..."


In [11]:
# merge with VA data
ny2=pd.merge(ny, usda, left_on='CountyId', right_on='FIPS', how='left')
print(ny.shape)
print(ny2.shape)

(62, 37)
(62, 43)


In [12]:
# what are the counts?
ny2['RUCC_2013'].value_counts()
#counties2['RUCC_2013'] = counties2['RUCC_2013'].astype(int)
#counties2['RUCC_2013'].value_counts()

1    20
2    12
4    10
6     9
3     6
7     3
5     1
8     1
Name: RUCC_2013, dtype: int64

In [13]:
# simplify the rural-urban indicator
ny2['metro']=ny2['RUCC_2013'].map({1:'urban', 
                                   2:'suburban',
                                   3:'suburban',
                                   4:'town',
                                   5:'town',
                                   6:'town',
                                   7:'rural',
                                   8:'rural',
                                   9:'rural'})
# sort as categories
ny2['metro'] = pd.Categorical(ny2['metro'], ['urban', 'suburban', 'town', 'rural'])
ny2['metro'].value_counts()

urban       20
town        20
suburban    18
rural        4
Name: metro, dtype: int64

In [14]:
# are there differences in commuting and transit patterns?
ny2.groupby('metro')[['MeanCommute', 'Drive','Carpool','Transit']].mean().sort_index()

,MeanCommute,Drive,Carpool,Transit
metro,,,,
urban,31.360000,64.155000,7.565000,17.355000
suburban,22.727778,79.533333,8.405556,2.133333
town,23.410000,80.005000,9.255000,0.855000
rural,22.500000,76.125000,8.925000,0.975000


In [15]:
# are there differences in income and poverty?
ny2.groupby('metro')[['Income', 'IncomePerCap','Unemployment', 'Poverty']].mean().sort_index()

,Income,IncomePerCap,Unemployment,Poverty
metro,,,,
urban,68292.000000,34977.55,6.285000,13.320000
suburban,56796.611111,29597.50,6.355556,13.827778
town,51203.100000,26612.80,6.715000,14.600000
rural,50733.250000,24562.50,8.350000,15.425000


In [16]:
# are there differences in race-ethnicity?
ny2.groupby('metro')[['White','Black','Hispanic','Asian','Native']].mean().sort_index()

,White,Black,Hispanic,Asian,Native
metro,,,,,
urban,66.550000,10.650,15.195000,5.335000,0.200000
suburban,86.061111,4.450,4.194444,2.711111,0.227778
town,89.930000,2.875,4.245000,0.845000,0.390000
rural,90.800000,2.425,2.475000,0.725000,1.825000


In [17]:
# are there differences in industry?
ny2.groupby('metro')[['Professional','Service','Office','Construction','Production']].mean().sort_index()

,Professional,Service,Office,Construction,Production
metro,,,,,
urban,38.650000,19.200000,23.320000,8.415000,10.425000
suburban,37.305556,18.944444,23.505556,9.022222,11.238889
town,32.460000,19.755000,21.975000,11.205000,14.590000
rural,31.225000,23.975000,20.100000,13.275000,11.375000


In [18]:
# Differences in sector?
ny2.groupby('metro')[['PrivateWork','PublicWork']].mean().sort_index()

,PrivateWork,PublicWork
metro,,
urban,78.675,15.420000
suburban,75.250,18.544444
town,73.020,19.480000
rural,65.850,25.100000


In [19]:
# display all counties
#pip install plotly-geo
import plotly.figure_factory as ff

values = ny2['Income']

fig = ff.create_choropleth(fips=ny2['FIPS'], 
                            values=values, 
                            scope=['NY'], 
                            county_outline={'color': 'rgb(255,255,255)', 'width': 0.5})
iplot(fig)

/opt/miniconda3/envs/prj6/lib/python3.10/site-packages/plotly/figure_factory/_county_choropleth.py:89: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/miniconda3/envs/prj6/lib/python3.10/site-packages/plotly/figure_factory/_county_choropleth.py:106: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/miniconda3/envs/prj6/lib/python3.10/site-packages/plotly/figure_factory/_county_choropleth.py:123: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/miniconda3/envs/prj6/lib/python3.10/site-packages/plotly/figure_factory/_county_choropleth.py:129: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/miniconda3/envs/prj6/lib/python3.10/site

/opt/miniconda3/envs/prj6/lib/python3.10/site-packages/plotly/figure_factory/_county_choropleth.py:359: ShapelyDeprecationWarning:

Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.

/opt/miniconda3/envs/prj6/lib/python3.10/site-packages/plotly/figure_factory/_county_choropleth.py:363: ShapelyDeprecationWarning:

Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.

/opt/miniconda3/envs/prj6/lib/python3.10/site-packages/plotly/figure_factory/_county_choropleth.py:368: ShapelyDeprecationWarning:

Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.

/opt/miniconda3/envs/prj6/lib/python3.10/site-packages/plotly/figure_factory/_county_chorop

In [20]:
# export
# va2.to_pickle('va-stats.pkl')

In [21]:
counties2['FIPS'].dtype
counties2['fips']=counties2['FIPS'].astype(str)

In [21]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    countiesx = json.load(response)
fig = go.Figure(go.Choroplethmapbox(geojson=countiesx, 
                                    locations=counties['fips'], 
                                    z=counties2['Income'],
                                    colorscale='Electric', 
                                    text=counties2['County'],
                                    zmin=0, 
                                    zmax=100000,
                                    marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=5.8, mapbox_center = {"lat": 38.0293, "lon": -79.4428})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()
iplot(fig)
# https://community.plot.ly/t/what-colorscales-are-available-in-plotly-and-which-are-the-default/2079

KeyError: 'fips'